In [1]:
import pandas as pd
from glob import glob
import sys
import os

project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.append(project_root)

from utils.functions import filter_similar_atmospheric_conditions

In [2]:


all_files = glob("../unprocessed_data/*parquet")

s1a2020 = pd.read_parquet(all_files[0])
s1a2021 = pd.read_parquet(all_files[1])
s1b2020 = pd.read_parquet(all_files[2])
s1b2021 = pd.read_parquet(all_files[3])

print(len(s1a2020), len(s1a2021), len(s1b2020), len(s1b2021))

720678 685889 736103 709018


In [3]:
def remove_sea_ice(df):
    df = df[df["class_1"] != "SI"]
    df = df[df["class_2"] != "SI"]
    df = df[df["class_3"] != "SI"]

    return df

s1a2020 = remove_sea_ice(s1a2020)
s1a2021 = remove_sea_ice(s1a2021)
s1b2020 = remove_sea_ice(s1b2020)
s1b2021 = remove_sea_ice(s1b2021)





In [7]:
wspd_bin = (3, 15)
L_bin = (-10e9, -0.1)

s1a2020 = filter_similar_atmospheric_conditions(s1a2020, wspd_range=wspd_bin, L_range=L_bin)
s1a2021 = filter_similar_atmospheric_conditions(s1a2021, wspd_range=wspd_bin, L_range=L_bin)

s1b2020 = filter_similar_atmospheric_conditions(s1b2020, wspd_range=wspd_bin, L_range=L_bin)
s1b2021 = filter_similar_atmospheric_conditions(s1b2021, wspd_range=wspd_bin, L_range=L_bin)

print(len(s1a2020) + len(s1a2021) + len(s1b2020) + len(s1b2021))


2006967


In [13]:
s1a2020.columns

Index(['value', 'wm_type', 'lonlat_center', 'tif_path', 'process_status',
       'class_1', 'prob_1', 'class_2', 'prob_2', 'class_3', 'prob_3',
       'all_prob', 'processing_time', 'filename', 'date_time', 'latitude',
       'longitude', 'L', 'wspd', 'wdir', 'sst', 'airt', 'p', 'rh', 'blh',
       'msdwswrf', 'msdwlwrf', 'air_sea_diff'],
      dtype='object')